In [24]:
import pandas as pd
import os
import numpy as np
df = pd.read_pickle(os.path.join('..','data_frame.pickle'))

In [48]:
# Iteration
small_df = df.iloc[49980:50019, :].copy()
grouped = small_df.groupby('artist')
print(type(grouped))
for name, group_df in grouped:
    print(name)
    print(group_df)
    #break

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
Frost, Sir Terry
                artist            title               medium    year   
id                                                                     
4704  Frost, Sir Terry        Blue Moon  Lithograph on paper  1952.0  \
4705  Frost, Sir Terry      Boat Shapes     Linocut on paper  1952.0   
4706  Frost, Sir Terry      Boat Shapes     Linocut on paper  1954.0   
4707  Frost, Sir Terry      Boat Shapes     Linocut on paper  1954.0   
4708  Frost, Sir Terry            Leeds    Drypoint on paper  1956.0   
4709  Frost, Sir Terry  Camping, Anduze     Etching on paper  1979.0   
4710  Frost, Sir Terry     Umea, Sweden     Etching on paper  1979.0   
4711  Frost, Sir Terry    Self-Portrait     Etching on paper  1980.0   

      acquisitionYear width height units  
id                                        
4704           1983.0   355    273    mm  
4705           1983.0   132    143    mm  
4706           1983.0   131    155 

In [26]:
# Aggregate # Mins
for name, group_df in small_df.groupby('artist'):
    min_year = group_df['acquisitionYear'].min()
    print (f"{name}: {min_year}".format())

Frost, Sir Terry: 1983.0
Phillips, Esq Tom: 1983.0
Wols: 1983.0


In [42]:
# Fill values function 
def fill_values(series):
    values_counted = series.value_counts()
    if values_counted.empty:
        return series
    most_frequent = values_counted.index[0]
    new_medium = series.fillna(most_frequent)
    return new_medium

# Transform function
def transform_df(source_df):
    group_dfs = []  
    for name, group_df in source_df.groupby('artist'):
        filled_df = group_df.copy()
        filled_df.loc[:, 'medium'] = fill_values(group_df['medium'])
        group_dfs.append(filled_df)
        new_df = pd.concat(group_dfs)
    return new_df

In [49]:
small_df.loc[[4704,16441], 'medium'] = np.nan

In [45]:
# Check the result
# small_df.loc[[4704,16441], 'medium'] = np.nan
# Equivalent of editing by hand

filled_df = transform_df(small_df)

In [50]:
# Built-ins
# Transform
grouped_mediums = small_df.groupby('artist')['medium']
small_df.loc[:, 'medium'] = grouped_mediums.transform(fill_values)


In [60]:
# Min
small_df.groupby('artist')["acquisitionYear"].agg(np.min)
small_df.groupby('artist')["acquisitionYear"].min()

artist
Frost, Sir Terry     1983.0
Phillips, Esq Tom    1983.0
Wols                 1983.0
Name: acquisitionYear, dtype: float64

In [54]:
# Filter
grouped_titles = small_df.groupby('title')
title_counts = grouped_titles.size().sort_values(ascending=False)

In [15]:
# Lamnda Filter
condition = lambda x: len(x.index) > 1
dup_titles_df =grouped_titles.filter(condition)
dup_titles_df.sort_values('title', inplace=True)